In [1]:
import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from pathlib import Path
from itertools import combinations
from functools import reduce

In [2]:
data_dir = Path.cwd() / '..' / 'data'
targets_file = data_dir / 'labels.csv'
similarity_matrix_file = data_dir / 'processed' / 'similarity.npy'
filtered_texts_listing_file = data_dir / 'filtered_texts_listing.txt'

In [3]:
def multilabel_iterative_stratification(dataset, subset_ratios):
    """Split multi-label dataset into disjoint subsets based on even distribution of labels"""
    subset_ratios = np.asarray(subset_ratios)
    k = len(subset_ratios)

    subset_counts = (subset_ratios * len(dataset)).round(decimals=2)

    labels = pd.DataFrame(data={'total':dataset.sum()}).sort_values('total')
    labels = labels.assign(count_index = range(0, len(labels)))

    label_subset_counts = np.matrix(labels.total.values).transpose() \
                          * np.matrix(subset_ratios)
    label_subset_counts = np.around(label_subset_counts, decimals=2)
    
    groups = pd.DataFrame(data={'group': -1}, index=dataset.index)
    remaining_documents = dataset.loc[groups.group == -1,:]
    while len(remaining_documents) > 0:
        label_counts = remaining_documents.sum()
        min_label = label_counts[label_counts > 0].idxmin()

        applicable_documents = remaining_documents.loc[
            remaining_documents.loc[:, min_label] > 0, :]
        for document_id, row_data in applicable_documents.iterrows():
            max_size_label = label_subset_counts[
                labels.count_index[min_label], ].max()
            max_subset_by_label = np.nonzero(
                label_subset_counts[labels.count_index[min_label], ] == max_size_label)[0]

            if len(max_subset_by_label) == 1:
                max_subset = max_subset_by_label[0]
            else:
                max_size_count = subset_counts[max_subset_by_label].max()
                max_subset_by_count = np.nonzero(
                    subset_counts == max_size_count)[0]

                if len(max_subset_by_count) == 1:
                    max_subset = max_subset_by_count[0]
                else:
                    max_subset = np.random.choice(max_subset_by_count, 1)[0]

            groups.loc[document_id, 'group'] = max_subset
            label_subset_counts[
                remaining_documents.loc[document_id, :].nonzero()[0],
                max_subset] -= 1
            subset_counts[max_subset] -= 1

        remaining_documents = dataset.loc[groups.group == -1,:]
        
    return groups

In [4]:
def calculate_prior(training_matrix, smoothing=1):
    m = len(training_matrix.coords['document_id'])

    prior_has_label = ((smoothing + training_matrix.sum('document_id')) / (2 * smoothing + m))
    prior_has_not_label = 1 - prior_has_label
    
    return (prior_has_label, prior_has_not_label)

In [5]:
def get_k_nearest_neighbors(instance, similarity_matrix, k):
    topk_idx = np.argpartition(
        similarity_matrix.sel(document_id_a=instance).values,
        -(k + 1))[-(k + 1):]

    not_instance = np.nonzero(
            (similarity_matrix.coords['document_id_b'][topk_idx] != instance).values)[0]
    topk_idx = topk_idx[not_instance]
    topk_idx = similarity_matrix.coords['document_id_b'][topk_idx]

    return topk_idx

In [6]:
def calculate_posterior(training_matrix, training_similarity, k, smoothing = 1):
    counting_membership_ids = []
    counting_membership_vectors = []
    for document_id in training_matrix.coords['document_id'].values:
        # for each training document, get k nearest neighbors
        topk_neighbors = get_k_nearest_neighbors(document_id, training_similarity, k)
        counting_membership_ids.append(document_id)
        counting_vector = training_matrix.sel(document_id=topk_neighbors.values).sum('document_id')
        counting_membership_vectors.append(counting_vector)

    counting_membership = xr.concat(
        counting_membership_vectors,
        pd.Index(
            counting_membership_ids,
            name='document_id'))

    has_label_counts = xr.DataArray(
        data=np.zeros((len(training_matrix.coords['label']), k + 1),
                      dtype='int64'),
        coords=[training_matrix.coords['label'].values, range(k + 1)],
        dims=['label', 'k'])

    has_not_label_counts = xr.DataArray(
        data=np.zeros((len(training_matrix.coords['label']), k + 1),
                      dtype='int64'),
        coords=[training_matrix.coords['label'].values, range(k + 1)],
        dims=['label', 'k'])

    for label in training_matrix.coords['label'].values:
        has_bin_counts = np.bincount(
            counting_membership.sel(label=label)[
                training_matrix.sel(label=label) == 1])

        padded_has_counts = np.pad(
            has_bin_counts,
            pad_width=(0, (k + 1) - len(has_bin_counts)),
            mode='constant',
            constant_values=(0, 0))

        has_label_counts.sel(label=label).values += padded_has_counts

        not_bin_counts = np.bincount(
            counting_membership.sel(label=label)[
                training_matrix.sel(label=label) == 0])

        padded_not_counts = np.pad(
            not_bin_counts,
            pad_width=(0, (k + 1) - len(not_bin_counts)),
            mode='constant',
            constant_values=(0, 0))

        has_not_label_counts.sel(label=label).values += padded_not_counts

    # posterior probability that target has # of neighbors with given label
    # conditional that it also has that label
    post_cond_has_label = (smoothing + has_label_counts) / (smoothing * (k + 1)
                                                           + has_label_counts.sum(dim='k'))

    # posterior probability that target has # of neighbors with given label
    # conditional that it does not have that label
    post_cond_not_label = (smoothing + has_not_label_counts) / (smoothing * (k + 1)
                                                                + has_not_label_counts.sum(dim='k'))
    
    return (post_cond_has_label, post_cond_not_label)

In [7]:
def evaluate_test_data(testing_matrix, priors, posteriors, testing_similarity, k):
    counting_membership_ids = []
    counting_membership_vectors = []
    for document_id in testing_matrix.coords['document_id'].values:
        # for each training document, get k nearest neighbors
        topk_neighbors = get_k_nearest_neighbors(document_id, testing_similarity, k)
        counting_membership_ids.append(document_id)
        counting_vector = testing_matrix.sel(document_id=topk_neighbors.values).sum('document_id')
        counting_membership_vectors.append(counting_vector)

    counting_membership = xr.concat(
        counting_membership_vectors,
        pd.Index(
            counting_membership_ids,
            name='document_id'))

    predictions = (priors[0] * posteriors[0].sel(k=counting_membership) 
                   > 
                   priors[1] * posteriors[1].sel(k=counting_membership)).astype('int64')

    predicted_ranking = (priors[0] * posteriors[0].sel(k=counting_membership)) /  (
        priors[0] * posteriors[0].sel(k=counting_membership)
        + priors[1] * posteriors[1].sel(k=counting_membership))
    
    return predictions, predicted_ranking

In [8]:
def hamming_loss(expected, predicted):
    matches = (expected != predicted)
    hamming_loss = matches.sum() / \
                   (len(matches.coords['document_id']) *
                    len(matches.coords['label']))
    return hamming_loss.values.item()

In [9]:
def one_error(testing_matrix, predictions, label_probs):
    max_label_idx = label_probs.argmax(dim='label')
    matches = predictions.isel(label=max_label_idx) \
              != testing_matrix.isel(label=max_label_idx)
    one_error = (matches.sum() / len(matches.coords['document_id']))
    return one_error.values.item()

In [10]:
def coverage(testing_matrix, ranking):
    total = np.where((testing_matrix == 1).transpose().values, ranking.values, 0).max(axis=0).sum()
    total -= len(testing_matrix.coords['document_id'])
    coverage = total / len(testing_matrix.coords['document_id'])
    
    return coverage

In [11]:
def split_subset_data(groups, subsets, dataset):
    subset_data = {}
    for subset in subsets:
        dataset_subset = dataset.loc[groups.group == subset]
        dataset_matrix_subset = xr.DataArray(dataset_subset.as_matrix(),
                      coords=[dataset_subset.index.values, dataset_subset.columns.values],
                      dims=['document_id', 'label'])

        subset_data[subset] = dataset_matrix_subset
        
    return subset_data

In [12]:
def join_selected_subsets(selected_subsets, subset_data, similarity_matrix):
    dataset_subsets = (subset_data[subset]
                       for subset in selected_subsets)
    dataset_matrix = xr.concat(dataset_subsets, dim='document_id')
    
    similarity_matrix_subset = similarity_matrix.sel(
        document_id_a=dataset_matrix.document_id.values,
        document_id_b=dataset_matrix.document_id.values)
    
    return dataset_matrix, similarity_matrix_subset

In [13]:
def convert_to_labels(instance, data_matrix):
    return list(data_matrix.coords['label'][data_matrix.sel(document_id=instance) == 1].values)

In [14]:
filtered_listing = pd.read_csv(filtered_texts_listing_file, names=['document_id'])
filtered_listing = filtered_listing.document_id.str.split('.').str.get(0).astype('int64')

dataset = pd.read_csv(targets_file, header=None, names=['id', 'subject'])
dataset.subject = dataset.subject.astype('category')
dataset = pd.get_dummies(dataset, prefix='', prefix_sep='').groupby('id').max()
dataset = dataset.loc[filtered_listing.values, :]

k = 10
subset_ratios = np.repeat(0.1, k)

similarity_matrix_raw = np.memmap(
    similarity_matrix_file,
    dtype='float32', mode='r',
    shape=(dataset.shape[0], dataset.shape[0]))

document_ids = filtered_listing.sort_values().values

similarity_matrix = xr.DataArray(
    data=similarity_matrix_raw,
    coords=[document_ids, document_ids],
    dims=['document_id_a', 'document_id_b'])

groups = multilabel_iterative_stratification(dataset, subset_ratios)
subsets = np.array(groups.group.unique())
subset_data = split_subset_data(groups, subsets, dataset)

In [15]:
rounds_hamming_loss = []
rounds_one_error = []
rounds_coverage = []

for round_id, test_subsets in enumerate(combinations(subsets, 1)):
    train_subsets = subsets[np.bitwise_not(np.isin(subsets, test_subsets))]
    
    training_matrix, training_similarity = join_selected_subsets(train_subsets, subset_data,
                                                                similarity_matrix)
    testing_matrix, testing_similarity = join_selected_subsets(test_subsets, subset_data,
                                                              similarity_matrix)

    priors = calculate_prior(training_matrix, smoothing=1)
    posteriors = calculate_posterior(training_matrix, training_similarity, smoothing=1, k=k)

    predictions, label_probs = evaluate_test_data(
        testing_matrix, priors, posteriors, testing_similarity, k)

    ordering = label_probs.argsort(axis=0).sel(label=slice(None, None, -1)) 
    ranking = ordering.argsort(axis=0) + 1
    
    rounds_hamming_loss.append(hamming_loss(testing_matrix, predictions))
    rounds_one_error.append(one_error(testing_matrix, predictions, label_probs))
    rounds_coverage.append(coverage(testing_matrix, ranking))

rounds_hamming_loss = np.array(rounds_hamming_loss)
rounds_one_error = np.array(rounds_one_error)
rounds_coverage = np.array(rounds_coverage)

In [16]:
rounds_hamming_loss.mean(), rounds_one_error.mean(), rounds_coverage.mean()

(0.02766217751184475, 0.3631220590574452, 11.947069346372976)